# Etapa 2: Pré-processamento de Dados e Treinamento do Modelo de IA

Este notebook realiza o pré-processamento dos dados simulados e treina um modelo de Machine Learning para classificar sequências de leituras do sensor ultrassônico.

---


## 1. Instalação e Importação de Bibliotecas

Instale e importe as bibliotecas necessárias para manipulação de dados, modelagem e visualização.

In [ ]:
# Instalar e importar bibliotecas necessárias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers




: 

## 2. Carregamento do Dataset

Carregue o arquivo de dados simulados gerado na Etapa 1.

In [ ]:
# Carregar o dataset
arquivo = 'dados_sensores_simulados.csv'
df = pd.read_csv(arquivo)
df.head()

## 3. Pré-processamento dos Dados

Agrupe os dados por `id_cenario` e crie janelas de tamanho fixo para alimentar o modelo sequencial.

In [ ]:
# Parâmetros de janelamento
janela = 30  # tamanho da janela (número de leituras por amostra)

X = []
y = []

for id_cenario, grupo in df.groupby('id_cenario'):
    distancias = grupo['distancia_cm'].values
    tipo = grupo['tipo_obstaculo'].iloc[0]
    # Gera janelas deslizantes
    for i in range(len(distancias) - janela + 1):
        X.append(distancias[i:i+janela])
        y.append(tipo)

X = np.array(X)
y = np.array(y)
print(f"Total de amostras: {X.shape[0]}")


In [ ]:
# Normalização dos dados
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X = X.reshape(-1, janela, 1)  # Formato para LSTM: (amostras, timesteps, features)


In [ ]:
# Divisão em treino, validação e teste
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)
print(f"Treino: {X_train.shape[0]}, Validação: {X_val.shape[0]}, Teste: {X_test.shape[0]}")


In [ ]:
# Conversão dos rótulos para one-hot encoding
# Corrige o offset dos rótulos para garantir que começam em 0
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_train_cat = keras.utils.to_categorical(le.transform(y_train), len(le.classes_))
y_val_cat = keras.utils.to_categorical(le.transform(y_val), len(le.classes_))
y_test_cat = keras.utils.to_categorical(le.transform(y_test), len(le.classes_))
num_classes = len(le.classes_)


## 4. Construção e Treinamento do Modelo

Crie e treine um modelo LSTM simples para classificação das sequências.

In [ ]:
# Modelo LSTM simples
model = keras.Sequential([
    layers.Input(shape=(janela, 1)),
    layers.LSTM(32, return_sequences=False),
    layers.Dense(32, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
# Callbacks
callbacks = [
    keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    keras.callbacks.ModelCheckpoint('melhor_modelo.h5', save_best_only=True)
]

# Treinamento
history = model.fit(
    X_train, y_train_cat,
    validation_data=(X_val, y_val_cat),
    epochs=30,
    batch_size=64,
    callbacks=callbacks
)


## 5. Avaliação e Salvamento do Modelo

Avalie o modelo no conjunto de teste e salve o modelo treinado.

In [ ]:
# Avaliação
loss, acc = model.evaluate(X_test, y_test_cat)
print(f"Acurácia no teste: {acc:.2%}")

# Salvamento
model.save('modelo_obstaculos_lstm.h5')
print("Modelo salvo como modelo_obstaculos_lstm.h5")


## 6. Visualização do Treinamento

Visualize a curva de acurácia e perda durante o treinamento.

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title('Acurácia')
plt.legend()
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Perda')
plt.legend()
plt.tight_layout()
plt.show()